In [25]:
import sys
import time
import dis

# Python bytecode

In [13]:
dis.dis(print("Hello, World!"))

Hello, World!
101           0 LOAD_GLOBAL              0 (compile)
              2 LOAD_FAST                1 (source)
              4 LOAD_FAST                2 (filename)
              6 LOAD_FAST                3 (symbol)
              8 LOAD_FAST                0 (self)
             10 LOAD_ATTR                1 (flags)
             12 LOAD_GLOBAL              2 (PyCF_ONLY_AST)
             14 BINARY_OR
             16 LOAD_CONST               1 (1)
    -->      18 CALL_FUNCTION            5
             20 RETURN_VALUE


### LOAD_GLOBAL 0: tells Python to look up the global object referenced by the name at index 0 of co_names (which is the print function) and push it onto the evaluation stack
### LOAD_CONST 16: takes the literal value at index 1 of co_consts and pushes it (the value at index 0 is the literal None, which is present in co_consts because Python function calls have an implicit return value of None if no explicit return statement is reached)
### CALL_FUNCTION 18: tells Python to call a function; it will need to pop one positional argument off the stack, then the new top-of-stack will be the function to call.

In [51]:
a=7
b=4
print(a+b)

11


In [32]:
dis.dis(print(a+b))

11
 52           0 LOAD_FAST                0 (x)
              2 LOAD_CONST               1 (None)
              4 COMPARE_OP               8 (is)
              6 POP_JUMP_IF_FALSE       22

 53           8 LOAD_GLOBAL              0 (distb)
             10 LOAD_FAST                1 (file)
             12 LOAD_CONST               2 (('file',))
             14 CALL_FUNCTION_KW         1
             16 POP_TOP

 54          18 LOAD_CONST               1 (None)
             20 RETURN_VALUE

 56     >>   22 LOAD_GLOBAL              1 (hasattr)
             24 LOAD_FAST                0 (x)
             26 LOAD_CONST               3 ('__func__')
             28 CALL_FUNCTION            2
             30 POP_JUMP_IF_FALSE       38

 57          32 LOAD_FAST                0 (x)
             34 LOAD_ATTR                2 (__func__)
             36 STORE_FAST               0 (x)

 59     >>   38 LOAD_GLOBAL              1 (hasattr)
             40 LOAD_FAST                0 (x)
            

In [52]:
dis.dis("{}") 

  1           0 BUILD_MAP                0
              2 RETURN_VALUE


In [53]:
dis.dis("dict()")

  1           0 LOAD_NAME                0 (dict)
              2 CALL_FUNCTION            0
              4 RETURN_VALUE


In [54]:
c = compile("a=a+1", "", "single")
c

<code object <module> at 0x7f7fb66d39d0, file "", line 1>

In [55]:
co = compile('2+2', '<none>', 'eval')
#co = compile("print('Hello, wrld!')", '<string>', 'eval')
co.co_code

b'd\x00S\x00'

### The result is a bytes literal which is prefixed with b'. 
### It is an immutable sequence of bytes and has a type of bytes.

In [56]:
for byte in co.co_code:
    print(byte, end=' ')

100 0 83 0 

# Optimization

## A. Lists vs Tuples


### 1. Lists over-allocate memory
Unlike lists, tuples do not use over-allocation. They are of fixed size and can store data more compactly.

In [19]:
a = tuple(range(10))

b = list(range(10))

In [20]:
a

(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)

In [21]:
a[3]

3

In [22]:
b

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [23]:
b[3]

3

In [30]:
sys.getsizeof(a)

4064

In [31]:
sys.getsizeof(b)

4568

### 2. Lists have faster append
Due to lists over-allocation, it reduces the cost of append operation as Python doesn’t need to allocate it memory at that time. Hence the append operation of a list is faster than tuple’s.

In [29]:
a = (1,2,3) #tuple
b = [1,2,3] #list

start = time.time()
for i in range(0,500):
    a += (4,)
tuple_time  = time.time() - start
print('Time for tuple - ', tuple_time)

start = time.time()
for i in range(0,500):
    b.append(4)
list_time = time.time() - start
print('Time for list - ', list_time)

if list_time < tuple_time:
    print('List is faster')
else:
    print('Tuple is faster')

Time for tuple -  0.001009225845336914
Time for list -  0.0002467632293701172
List is faster


### 3. Copy
 Since tuples are immutable, they do not have to be copied. Rather, Python just refers the memory address of the old tuple to the new one.

In [32]:
a = (1,2,3) #tuple
b = [1,2,3] #list

In [33]:
print(a, b)

(1, 2, 3) [1, 2, 3]


In [34]:
c = a
c is a

True

In [35]:
d = b
d is b

True

In [36]:
c = c + (7,)

In [37]:
d.append(7)

In [38]:
c, d

((1, 2, 3, 7), [1, 2, 3, 7])

In [39]:
print(a, b)

(1, 2, 3) [1, 2, 3, 7]


## B. Dictionary vs List

In [40]:
list1 = [4, 0.22, 'Hello', [1, 2, 3], -2.5, 0.22]
dict1 = {'key1': [2.3,3], 'key2': "smthng", '34': 33}

In [41]:
def find_number_in_list(lst, number):
    if number in lst:
        return True
    else:
        return False

In [42]:
def find_number_in_dict(dct, number):
    if number in dct.keys():
        return True
    else:
        return False

In [43]:
short_list = list(range(100))
long_list = list(range(100000000))

In [44]:
short_dict = {x:x*5 for x in range(1,100)}
long_dict = {x:x*5 for x in range(1,100000000)}

In [45]:
%timeit find_number_in_list(short_list, 99)

1.84 µs ± 604 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [46]:
%timeit find_number_in_list(long_list, 99999999)

1.36 s ± 62.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [47]:
%timeit find_number_in_dict(short_dict, 99)

168 ns ± 13.3 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [48]:
%timeit find_number_in_dict(long_dict, 99999999)

220 ns ± 35.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [49]:
sys.getsizeof(long_list)/(1024*1024) #MB

762.9395065307617

In [50]:
sys.getsizeof(long_dict)/(1024*1024) #MB

5120.00008392334

In [ ]:
#10Hz sensor: 1 month of data: 25.920.000 records, 4 month: 103.680.000